<a href="https://colab.research.google.com/github/akunamatata87/TestBot/blob/master/BTC_Risk_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##########################################################
#          Last Update: 18 Dec 2021                      #
#   Add price prediction according to specific risk      #
##########################################################    

##########################################################
#                   How to run                           #
#   Just press play on the left side of this page        #
#   and wait for a minute, the graphs will be displayed  #
#           at the bottom of the page                    #
##########################################################    

!pip install quandl 
!pip install plotly --upgrade
!pip install yfinance
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import quandl
import yfinance as yf

# Download historical data from Quandl
df = quandl.get('BCHAIN/MKPRU', api_key='FYzyusVT61Y4w65nFESX').reset_index()

# Convert dates to datetime object for easy use
df['Date'] = pd.to_datetime(df['Date'])

# Sort data by date, just in case
df.sort_values(by='Date', inplace=True)

# Only include data points with existing price
df = df[df['Value'] > 0]

# Get the last price against USD
btcdata = yf.download(tickers='BTC-USD', period='1d', interval='1m')

# Append the latest price data to the dataframe
df.loc[df.index[-1]+1] = [date.today(), btcdata['Close'].iloc[-1]]
df['Date'] = pd.to_datetime(df['Date'])

# Calculate the `Risk Metric`
df['MA'] = df['Value'].rolling(374, min_periods=1).mean().dropna()
df['Preavg'] = (np.log(df.Value) - np.log(df['MA'])) * df.index**.395

# Normalization to 0-1 range
df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())

# Predicting the price according to risk level
price_per_risk = {
    round(risk, 2):round(np.exp(
        (risk * (df['Preavg'].cummax().iloc[-1] - (df['Preavg'].cummin().iloc[-1])) + df['Preavg'].cummin().iloc[-1]) / df.index[-1]**.395 + np.log(df['MA'].iloc[-1])
    ))
    for risk in np.arange(0.0, 1.0, 0.1)
}

# Exclude the first 1000 days from the dataframe, because it's pure chaos
df = df[df.index > 1000]

# Title for the plots
AnnotationText = f"Updated: {btcdata.index[-1]} | Price: {round(df['Value'].iloc[-1])} | Risk: {round(df['avg'].iloc[-1], 2)}"

# Plot BTC-USD and Risk on a logarithmic chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Add BTC-USD and Risk data to the figure
fig.add_trace(go.Scatter(x=df['Date'], y=df['Value'], name='Price', line=dict(color='gold')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['avg'],   name='Risk',  line=dict(color='white')), secondary_y=True)

# Add green (`accumulation` or `buy`) rectangles to the figure
opacity = 0.2
for i in range(5, 0, -1):
    opacity += 0.05
    fig.add_hrect(y0=i*0.1, y1=((i-1)*0.1), line_width=0, fillcolor='green', opacity=opacity, secondary_y=True)

# Add red (`distribution` or `sell`) rectangles to the figure
opacity = 0.2
for i in range(6, 10):
    opacity += 0.1
    fig.add_hrect(y0=i*0.1, y1=((i+1)*0.1), line_width=0, fillcolor='red', opacity=opacity, secondary_y=True)

fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_yaxes(title='Risk', type='linear', secondary_y=True, showgrid=True, tick0=0.0, dtick=0.1, range=[0, 1])
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot BTC-USD colored according to Risk values on a logarithmic chart
fig = px.scatter(df, x='Date', y='Value', color='avg', color_continuous_scale='jet')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot Predicting BTC price according to specific risk
fig = go.Figure(data=[go.Table(
    header=dict(values=['Risk', 'Price'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[list(price_per_risk.keys()), list(price_per_risk.values())],
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])
fig.update_layout(width=500, height=500, title={'text': 'Price according to specific risk', 'y': 0.9, 'x': 0.5})
fig.show()

     |████████████████████████████████| 26.5 MB 1.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 6.4 MB 37.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
ERROR: Operation cancelled by user


ModuleNotFoundError: ignored